In [1]:
import pandas as pd
import numpy as np

In [101]:
seasons = list(range(2009,2019))

# merge seasons into one complete df
df = pd.DataFrame()
for season in seasons:
    path = './data/nflscrapR-data/play_by_play_data/regular_season/reg_pbp_'+str(season)+'.csv'
    season_df = pd.read_csv(path)
    df = pd.concat([df, season_df], axis=0)
    path = './data/nflscrapR-data/play_by_play_data/post_season/post_pbp_'+str(season)+'.csv'
    post_season_df = pd.read_csv(path)
    df = pd.concat([df, post_season_df], axis=0)
print(df.shape)

print(df['play_type'].unique())

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42,166,167,174,175,178,179,182,183,188,189,190,191,203,204,205,231,232,233,238,240,241) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (37,38,39,42,46,47,166,167,168,169,172,173,174,175,176,177,178,179,182,183,184,185,188,189,190,191,192,193,194,195,198,199,203,204,205,224,225,226,227,231,232,233,238,240,241,244,245,246,250) have mixed types. Speci

(473782, 256)
['kickoff' 'pass' 'run' 'punt' 'field_goal' 'no_play' nan 'extra_point'
 'qb_kneel' 'qb_spike']


In [90]:
# df = df.sort_values(by=['epa'],ascending=True)
# top_5 = df.head(5)
# for index, row in top_5.iterrows():
#     print("Description",row['desc'],"\n","EPA",np.round(row['epa'],1))
# df = df.sort_values(by=['epa'],ascending=False)
# top_5 = df.head(5)
# for index, row in top_5.iterrows():
#     print("Description",row['desc'],"\n","EPA",np.round(row['epa'],1))

Description (12:14) (No Huddle, Shotgun) M.Vick to WAS 10 for -6 yards (R.Kerrigan). FUMBLES (R.Kerrigan), touched at WAS 10, RECOVERED by WAS-D.Hall at WAS 25. D.Hall for 75 yards, TOUCHDOWN. Lateral batted by 91 - Kerrigan The Replay Assistant challenged the backward pass ruling, and the play was Upheld. 
 EPA -12.8
Description (11:14) (Shotgun) J.Tuel pass short right intended for T.Graham INTERCEPTED by S.Smith at KC 0. S.Smith for 100 yards, TOUCHDOWN. 
 EPA -12.7
Description (6:05) (Shotgun) M.Cassel sacked at DEN 25 for -17 yards (M.Haggan). FUMBLES (M.Haggan), RECOVERED by DEN-J.Hunter at DEN 25. J.Hunter for 75 yards, TOUCHDOWN. 
 EPA -12.1
Description (2:51) M.Vick pass short left intended for B.Celek INTERCEPTED by P.Robinson at NO 1. P.Robinson for 99 yards, TOUCHDOWN. PENALTY on PHI-M.Vick, Low Block, 15 yards, enforced between downs. The penalty for a low block during the interception return will be assessed on the ensuing kickoff. 
 EPA -12.1
Description (7:50) (Shotgun)

In [179]:
print(len(df))
passes = df.loc[df.play_type=='pass']
print(len(passes))

473782
197305


In [191]:
# passes = passes.loc[passes.passer_player_name=='M.Sanu'] # for individual players
passes = passes.loc[~passes.desc.str.contains('aborted|Aborted')]
passes = passes.loc[~passes.desc.str.contains('Field Goal|Kick formation|Punt formation|Field Goal formation')]
passes = passes.loc[~passes.desc.str.contains('TWO-POINT CONVERSION ATTEMPT')]
passes = passes.loc[~passes.desc.str.contains('play under review|PENALTY')]
passes = passes.loc[(passes.wp >= 0.1)&(passes.wp <= 0.9)]
print(len(passes))


8
2.075336279173448


In [181]:
# add column that counts how many times a player throws the ball in the dataset - RB's and WR's don't throw much obv
passes['passer_count'] = passes.groupby(['passer_player_name'])['epa'].transform("count")

In [184]:
new_pass = passes.loc[passes.passer_count <= 30]
new_pass2 = passes.loc[passes.desc.str.contains('Flea|flea')]
new_pass = pd.concat([new_pass,new_pass2],axis=0)
print(len(new_pass))

66


In [185]:
# doesn't catch everyone
print(len(new_pass))

blacklist = ["D.Fales","M.Brunell","B.Croyle","R.Smith","T.Bouman","T.Thigpen","C.Shaw","S.Mannion","C.Simms","J.O'Sullivan","Shaun.Hill","P.White","M.Simms","B.St. Pierre","R.Bartel","K.Allen","J.Palmer","L.Brown","M.Kafka","Ale.Smith","J.Losman","Matt.Moore","Dom.Davis","L.Thomas","R.Nassib","T.Boykin","C.Jones","C.Cook","J.Callahan","N.Sudfeld","R.Griffin III","K.Lauletta","G.Gilbert","J.Dobbs","J.Rudock","A.Tanney","Jo.Freeman","S.Renfree","Mat.Moore"]
new_pass = new_pass.loc[~new_pass.passer_player_name.isin(blacklist)]
new_pass = new_pass.loc[~new_pass.passer_player_name.str.contains('3rd QB')]
runs = runs.loc[~runs.desc.str.contains('lateral|Lateral|reverse|Reverse')]

print(len(new_pass))

66
66


In [186]:
for index, row in new_pass.iterrows():
    print(row['passer_player_name'], row['desc'])

T.Brady (10:02) T.Brady pass deep middle to R.Moss for 40 yards, TOUCHDOWN. flea flicker NE 81-Moss 136th career receiving TD (3rd best all-time) and 137th career TD (6th best all-time).
M.Bulger (7:01) M.Bulger pass deep middle to D.Avery to IND 14 for 50 yards (A.Bethea). Flea-flicker
M.Sanchez (6:45) NYJ #6 Sanchez handed off to #20 Jones then passed backward to Sanchez (Flea-flicker). M.Sanchez pass incomplete deep right to B.Edwards.
T.Brady (9:40) M.LeVoir reported in as eligible.  T.Brady pass incomplete deep left to R.Moss. flea flicker
M.Stafford (9:43) M.Stafford pass incomplete short right to C.Johnson. {Flea flicker }
B.Roethlisberger (13:21) Flea-flicker play, hand-off to #34 Mendenhall. B.Roethlisberger pass short left to R.Mendenhall to PIT 35 for 11 yards (N.Barnett).
D.Brees (6:57) D.Brees pass deep left to D.Henderson for 44 yards, TOUCHDOWN. Flea-flicker: #9 Brees handed off to #23 Thomas, who lateralled back to Brees, throwing to #19 Henderson for the touchdown.  Pa

In [188]:
print(new_pass.epa.mean())
print(new_pass.epa.std())
print(passes.epa.mean())
print(passes.epa.std())
print(len(new_pass))
print(len(passes))

0.5571306900427668
1.7737544599026598
0.49075287516501864
1.6336522793953403
66
150959


In [162]:
runs = df.loc[df.play_type=='run']
runs = runs.loc[~runs.desc.str.contains('PENALTY|Aborted|Penalty|TWO-POINT CONVERSION ATTEMPT')]
runs = runs.loc[~runs.desc.str.contains('RECOVERED by WAS-D.Hall at WAS 25. D.Hall for 75 yards,')] # play i noticed that wasn't a trick play
runs = runs.loc[(runs.wp >= 0.1)&(runs.wp <= 0.9)]

In [163]:
print(len(runs))
new_runs = runs.loc[runs.desc.str.contains('lateral|Lateral|reverse|Reverse')]
runs = runs.loc[~runs.desc.str.contains('lateral|Lateral|reverse|Reverse')] # remove trick plays from control dataset
print(len(new_runs))

106811
180


In [164]:
for index, row in new_runs.iterrows():
    print(row['rusher_player_name'], row['desc'])

C.Ochocinco (4:25) C.Ochocinco right end to CIN 41 for 8 yards (C.Bailey). {Double reverse, Palmer to Coles to Ochocinco}
B.Smith (8:38) B.Smith left end to NE 15 for 7 yards (S.Springs, J.Green). #6 Sanchez handed off to #29 Washington who handed off on the reverse to #16 B.Smith
T.Owens (13:20) T.Owens left end to TB 41 for 13 yards (G.Adams, B.Ruud). Reverse, handoff from F.Jackson.
R.Parrish (1:58) R.Parrish right end to BUF 36 for 9 yards (B.Ruud, W.Allen). Reverse, handoff from T.Edwards.
M.Clayton (8:13) (No Huddle, Shotgun) M.Clayton right end to CLE 28 for 12 yards (M.Adams). Clayton carries ball on pitch from Flacco on reverse.
J.Edelman (13:07) J.Edelman left end pushed ob at NE 38 for 5 yards (T.DeCoud). reverse
R.Bush (5:36) R.Bush right end to BUF 36 for 14 yards (M.Stroud). Reverse, handoff from D.Brees. R.Bush lined up at WR.
C.Johnson (6:39) #81 Johnson took the reverse handoff from #34 Smith C.Johnson left end to CHI 37 for 12 yards (D.Manning).
M.Bradley (9:14) M.Bra

In [195]:
print(new_runs.epa.count())
print(new_runs.epa.std())

180
1.222616195386599


In [190]:
print(runs.epa.mean())
print(runs.epa.std())

0.5337730791878021
0.9961841535127494
